In [1]:
from keras.layers import Input, Dense
from keras.layers.core import Reshape
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.convolutional import Convolution2D as Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers import merge
from keras.optimizers import Adadelta, RMSprop
import os
import os.path
import numpy as np
from PIL import Image
from numpy import * 
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
import scipy.misc
import cv2

/home/shreyas/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/shreyas/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def model():

    input_img = Input(shape=(128,128,1))
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
      
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
       
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    
    up6 = UpSampling2D((2,2))(conv5)
    up6 = merge([up6, conv4], mode='concat', concat_axis=3)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    
    up7 = UpSampling2D((2,2))(conv6)
    up7 = merge([up7, conv3], mode='concat', concat_axis=3)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    
    up8 = UpSampling2D((2,2))(conv7)
    up8 = merge([up8, conv2], mode='concat', concat_axis=3)
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    
    up9 = UpSampling2D((2,2))(conv8)
    up9 = merge([up9, conv1], mode='concat', concat_axis=3)
    conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
       
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(conv9)
    
    ada=Adadelta(lr=5.0, rho=0.95, epsilon=1e-08, decay=0.001)
    rms=RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.001)
    
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(loss='mean_squared_error', optimizer=rms)
    return autoencoder

In [3]:
autoencoder=model()

original_matrix=[]
transformed_matrix=[]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/shreyas/.local/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarni

In [4]:
path1="Data"

In [5]:
for i in range(1,123):
    path_major=path1+'/'+str(i)
    for j in range(1,10):
        img=array(Image.open(path_major+"/"+str(j)+".jpg"))
        #print shape(img)
        img = cv2.cvtColor( img, cv2.COLOR_RGB2GRAY )
        img=img.reshape(128,128,1)
        original_matrix.append(img)
        transformed_matrix.append(img)
#     for j in range(1,10):
#         img=array(Image.open(path_major+"/"+str(j)+"_.jpg"))
#         img = cv2.cvtColor( img, cv2.COLOR_RGB2GRAY )
#         img=img.reshape(128,128,1)
#         transformed_matrix.append(img)

In [6]:
data,Label = shuffle(original_matrix,transformed_matrix, random_state=2)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, Label, test_size=0.2, random_state=2)
X_train = array(X_train)
y_train = array(y_train)
X_test = array(X_test)
y_test = array(y_test)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(878, 128, 128, 1)
(878, 128, 128, 1)
(220, 128, 128, 1)
(220, 128, 128, 1)


In [8]:
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.

y_train = y_train.astype('float32') / 255.
y_test = y_test.astype('float32') / 255.
saveDir = 'sample/'

In [1]:
for epoch in range(1,100):
    
    train_X,train_Y=shuffle(x_train,y_train)
    print ("Epoch is: %d\n" % epoch)
    batch_size=128
    print ("Number of batches: %d\n" % int(len(train_X)/batch_size))
    num_batches=int(len(train_X)/batch_size)
    for batch in range(num_batches):    
        batch_train_X=train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
        batch_train_Y=train_Y[batch*batch_size:min((batch+1)*batch_size,len(train_Y))]
        loss=autoencoder.train_on_batch(batch_train_X,batch_train_Y)
        print ('epoch_num: %d batch_num: %d loss: %f\n' % (epoch,batch,loss))

    autoencoder.save_weights("fabric_autoen.h5")
    if(epoch%5==0):
        x_test,y_test=shuffle(x_test,y_test)
        decoded_imgs=autoencoder.predict(x_test[:2])
        temp = np.zeros([128, 128*3,3])
        temp[:, :128,:1] = x_test[0,:,:,:1]
        temp[:, 128:128*2,:1] = y_test[0,:,:,:1]
        temp[:, 128*2:,:1] = decoded_imgs[0,:,:,:1]
        temp[:,:,1]=temp[:,:,0]
        temp[:,:,2]=temp[:,:,0]
        temp = temp*255
        scipy.misc.imsave(saveDir + str(epoch/5) + ".jpg", temp)

NameError: name 'shuffle' is not defined